In [90]:
import pandas as pd
import requests
import json
import re

In [91]:
uni_name = "uni"
Employability = "ind_27"
Learning_Experience = "ind_2054337"
Diversity_Internationalisation = "ind_2054338"
Research = "ind_2054339"
one, two, three, four, five = [],[],[],[],[]
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36"}

In [92]:
response = requests.get("https://www.topuniversities.com/sites/default/files/qs-rankings-data/en/3738856_indicators.txt?1620211931?v=1620413640389",headers=headers,timeout=5)
qs_usa_dataset = json.loads(response.text)["data"]
for i in range(len(qs_usa_dataset)):
    one.append(re.match('.*class="uni-link">(.*)</a></div></div>', qs_usa_dataset[i][uni_name]).group(1))
    two.append(float(re.match('.*class="td-wrap-in">(.*)</div></div>', qs_usa_dataset[i][Employability]).group(1)))
    three.append(float(re.match('.*class="td-wrap-in">(.*)</div></div>', qs_usa_dataset[i][Learning_Experience]).group(1)))
    four.append(float(re.match('.*class="td-wrap-in">(.*)</div></div>', qs_usa_dataset[i][Diversity_Internationalisation]).group(1)))
    five.append(float(re.match('.*class="td-wrap-in">(.*)</div></div>', qs_usa_dataset[i][Research]).group(1)))

In [93]:
df = pd.DataFrame({
    "name":one,
    "Employability":two,
    "Learning_Experience":three,
    "Diversity_Inter":four,
    "Research":five
})

In [94]:
df.head()

,name,Employability,Learning_Experience,Diversity_Inter,Research
0,Harvard University,100.0,96.8,76.0,100.0
1,Stanford University,83.4,96.8,82.0,93.5
2,Massachusetts Institute of Technology (MIT),75.4,96.8,80.0,90.6
3,"University of California, Berkeley (UCB)",57.9,81.4,78.0,91.7
4,"University of California, Los Angeles (UCLA)",57.7,90.5,84.0,78.7


In [97]:
df.to_excel("qsrank.xlsx")